## Expense Forecaster

-- Body --

## Setup

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler

sns.set_style("whitegrid")
sns.set_palette("muted")
sns.set_context("notebook")

## Dataset

This project uses the "Personal Budget Transactions Dataset" dataset, which is publicly available on [Kaggle](https://www.kaggle.com/datasets/ismetsemedov/personal-budget-transactions-dataset). This dataset contains personal budget transaction records collected over a period of ~3 years.

In [2]:
df = pd.read_csv('expenses_data.csv')

In [3]:
df.shape

(4597, 3)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4597 entries, 0 to 4596
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      4597 non-null   object 
 1   category  4597 non-null   object 
 2   amount    4597 non-null   float64
dtypes: float64(1), object(2)
memory usage: 107.9+ KB


In [5]:
df.isnull().sum()

date        0
category    0
amount      0
dtype: int64

No null values, which makes it easier for us.

## Preprocessing

In [6]:
df

,date,category,amount
0,2022-07-06 05:57:10 +0000,Restuarant,5.50
1,2022-07-06 05:57:27 +0000,Market,2.00
2,2022-07-06 05:58:12 +0000,Coffe,30.10
3,2022-07-06 05:58:25 +0000,Market,17.33
4,2022-07-06 05:59:00 +0000,Restuarant,5.50
...,...,...,...
4592,2025-03-09 18:16:33 +0000,Market,2.88
4593,2025-03-10 02:50:47 +0000,Transport,0.50
4594,2025-03-10 13:25:19 +0000,Transport,0.50
4595,2025-03-10 14:21:57 +0000,Market,7.50


As we can see, the date is in a timestamp format, from which we will extract each individual component for better understanding.
We will also use the OneHotEncoder() function to convert the categories into a numerical format that can be understood by our models.

In [7]:
df['category'].unique()

array(['Restuarant', 'Market', 'Coffe', 'Transport', 'Other', 'Phone',
       'Communal', 'Clothing', 'Motel', 'Travel', 'Rent Car', 'Sport',
       'Events', 'Learning', 'Health', 'Taxi', 'Business lunch',
       'Film/enjoyment', 'Tech', 'joy', 'Fuel', 'business_expenses'],
      dtype=object)

We can also see variations in the naming of categories, such as 'joy', and 'business_expenses'. We will fix them for better consistency.

Let us now create a new dataframe to contain all the processed numerical data.

In [8]:
df["date"] = pd.to_datetime(df["date"]) #To convert the column from object type to datetime type

category_replacements = {
    'Restuarant': 'Restaurant',
    'Coffe': 'Coffee',
    'joy': 'Joy',
    'business_expenses': 'Business Expenses'
}

df['category'] = df['category'].replace(category_replacements)

In [9]:
df_date = pd.DataFrame({
    "year": df["date"].dt.year,
    "month": df["date"].dt.month,
    "day": df["date"].dt.day,
    "hour": df["date"].dt.hour,
    "minute": df["date"].dt.minute,
    "weekday": df["date"].dt.weekday
})

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

encoded_values = encoder.fit_transform(df[['category']])
new_cols = encoder.get_feature_names_out(['category'])

df_encoded = pd.DataFrame(encoded_values, columns=new_cols, index=df.index)

df_numeric = pd.concat([df_date, df_encoded, df.drop(columns=['date','category',])], axis=1)

In [16]:
df_numeric

,year,month,day,hour,minute,weekday,category_Business Expenses,category_Business lunch,category_Clothing,category_Coffee,...,category_Other,category_Phone,category_Rent Car,category_Restaurant,category_Sport,category_Taxi,category_Tech,category_Transport,category_Travel,amount
0,2022,7,6,5,57,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.50
1,2022,7,6,5,57,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00
2,2022,7,6,5,58,2,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.10
3,2022,7,6,5,58,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.33
4,2022,7,6,5,59,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4592,2025,3,9,18,16,6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.88
4593,2025,3,10,2,50,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.50
4594,2025,3,10,13,25,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.50
4595,2025,3,10,14,21,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.50


We should also scale the amount column for scale sensitive models.

In [21]:
scaler = StandardScaler()

df_numeric['amount'] = scaler.fit_transform(df_numeric[['amount']])


In [23]:
df_numeric

,year,month,day,hour,minute,weekday,category_Business Expenses,category_Business lunch,category_Clothing,category_Coffee,...,category_Other,category_Phone,category_Rent Car,category_Restaurant,category_Sport,category_Taxi,category_Tech,category_Transport,category_Travel,amount
0,2022,7,6,5,57,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.133180
1,2022,7,6,5,57,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.179373
2,2022,7,6,5,58,2,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.191486
3,2022,7,6,5,58,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022950
4,2022,7,6,5,59,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.133180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4592,2025,3,9,18,16,6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.167759
4593,2025,3,10,2,50,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.199169
4594,2025,3,10,13,25,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.199169
4595,2025,3,10,14,21,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.106785


We can now start analyzing the data!

## Exploratory Data Analysis (EDA)